In [2]:
from ase import Atoms
from ase.build import rotate
from ase.geometry import distance
from ase.build import hcp0001
from ase.io import write
import numpy as np
from ase.constraints import FixAtoms
import copy
import os
from ase.geometry import get_distances
from ase.calculators.lj import LennardJones 
from ase.calculators import emt

In [30]:
from openbabel import pybel
import numpy as np
import ase
from ase import Atoms
from ase.io import write
from ase.constraints import FixAtoms
smiles = '[H]'  

# 使用Openbabel创建分子对象
mol = pybel.readstring("smi", smiles)
mol.make3D(forcefield='mmff94', steps=100) 

# 创建ASE的Atoms对象
geo = Atoms()
geo.set_cell(np.array([[16.0, 0.0, 0.0], [0.0, 17.0, 0.0], [0.0, 0.0, 18.0]]))
geo.set_pbc((True, True, True))

# 将Openbabel分子对象添加到ASE的Atoms对象中
for atom in mol:
    atom_type = atom.atomicnum
    atom_position = np.array([float(i) for i in atom.coords])
    geo.append(atom_type)
    geo.positions[-1] = atom_position

# 调整分子位置
free_radical_position=geo.positions[0]
print('the position of Free Radical is',free_radical_position)
geo.translate(-free_radical_position)
print('the position of Free Radical is',free_radical_position)
# 保存为traj格式
write('output_ase.xyz', geo, format='xyz')

the position of Free Radical is [ 1.0189777  -0.09530499  0.02359672]
the position of Free Radical is [0. 0. 0.]


In [ ]:
#CatNet molecule to SMILES==>[H]C([H])([H])O to CH3O
import re
CNmol='[H]C([H])([O])O'
std_SMILES='C[O]'

def add_brackets_around_letters(s):
    # 使用正则表达式替换不在[]中的字母字符，前后添加[]
    result = re.sub(r'(?<!\[)([a-zA-Z])(?!\])', r'[\g<1>]', s)
    return result

# exampel
input_string = CNmol
modified_string = add_brackets_around_letters(input_string)
print("修改后的字符串:", modified_string)





In [ ]:
def read_file_line_by_line(file_path):
    mol_list=[]
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            string = line.strip()  # 打印或存储每行，去除换行符
            mol_list.append(string)
        mol_list.pop(0)
    return mol_list
# 使用函数
file_path = 'mollist.txt'  # 替换为你的文件路径
list1 = read_file_line_by_line(file_path)
print(list1)

In [ ]:
from rdkit import Chem
import numpy as np

def smiles_to_chemical_graph(smiles):
    # 从SMILES字符串创建分子对象
    mol = Chem.MolFromSmiles(smiles)
    # 添加氢原子
    mol = Chem.AddHs(mol)
    # 获取原子数量
    num_atoms = mol.GetNumAtoms()
    # 初始化原子种类列表
    atom_types = []
    # 初始化边列表，包含原子编号
    edges = []
    # 初始化邻接矩阵
    adjacency_matrix = np.zeros((num_atoms, num_atoms), dtype=int)
    # 遍历分子中的所有原子
    for i, atom in enumerate(mol.GetAtoms()):
        # 获取原子的符号（如'C'表示碳，'O'表示氧，'H'表示氢）
        atom_types.append((i, atom.GetSymbol()))
    # 遍历分子中的所有键
    for bond in mol.GetBonds():
        # 获取键的起始和结束原子索引
        begin_atom_idx = bond.GetBeginAtomIdx()
        end_atom_idx = bond.GetEndAtomIdx()
         # 将边添加到边列表中
        edges.append((begin_atom_idx, end_atom_idx))
        # 获取键的类型（单键、双键、三键或芳香键），并转换为整数
        bond_type = bond.GetBondType()
        # 在邻接矩阵中标记成键关系
        # 单键用1表示，双键用2表示，三键用3表示，芳香键用4表示
        bond_value = {
            Chem.BondType.SINGLE: 1,
            Chem.BondType.DOUBLE: 2,
            Chem.BondType.TRIPLE: 3,
            Chem.BondType.AROMATIC: 4
        }.get(bond_type, 1)  # 默认为单键
        adjacency_matrix[begin_atom_idx, end_atom_idx] = bond_value
        adjacency_matrix[end_atom_idx, begin_atom_idx] = bond_value  # 无向图，需要对称标记
    return atom_types, adjacency_matrix,edges#list,array,list


def count_nonzero_in_row(matrix):
    non_zero_counts = np.count_nonzero(matrix,axis=1)
    return non_zero_counts
def check_atoms_bonds(atom_types,bondarray):
    free_radical_position_list=[]
    for i in range(len(atom_types)):
        atom = atom_types[i]
        bonds = bondarray[i]
        if atom[1] == 'C' and bonds < 4:
            free_radical_position_list.append(atom)
        elif atom[1] == 'N' and bonds < 3:
            free_radical_position_list.append(atom)
        elif atom[1] == 'O' and bonds < 2:
            free_radical_position_list.append(atom)
        elif atom[1] == 'H' and bonds < 1:
            free_radical_position_list.append(atom)
        else:
            pass
    return free_radical_position_list



In [ ]:
def enumerate_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    smi = Chem.MolToSmiles(mol,doRandom=False,canonical=False)
    return smi
smiles = '[H][C]([H])([O])[O]'
smi_use = enumerate_smiles(smiles)
print(smi_use)

In [ ]:
# 示例SMILES字符串
smiles = "[H][C]([H])([O])[O]"  
smiles = enumerate_smiles(smiles)
atom_types, adjacency_matrix,edges = smiles_to_chemical_graph(smiles)

print("原子种类:", atom_types)
print("邻接矩阵:")
print(adjacency_matrix)
print('bonds:',edges)
bondlist = count_nonzero_in_row(adjacency_matrix)
print(bondlist)
freelist=check_atoms_bonds(atom_types,bondlist)
print(freelist)


In [ ]:
from rdkit import Chem
from pprint import pprint
# create a mol object
mol = Chem.MolFromSmiles('[H][C]([H])C')
# Generate 100 random SMILES
smis = []
for i in range(1):
    smis.append(Chem.MolToSmiles(mol,doRandom=True,canonical=False))
# remove duplicates
smis_set = list(set(smis))
smi_use = smis_set[0]
pprint(smi_use) # output order will be random; doctest skipped

In [ ]:
# Create an rdkit mol object
from rdkit import Chem
mol = Chem.MolFromSmiles('[H][C]([H])([O])[O]')
mol = Chem.AddHs(mol)
# Iterate over the atoms
for atom in mol.GetAtoms():
    # For each atom, set the property "atomNote" to a index+1 of the atom
    atom.SetProp("molAtomMapNumber", str(atom.GetIdx()))

mol

In [ ]:
from rdkit import Chem

# 创建分子对象，这里以SMILES格式为例
smiles = '[CH2][CH3]'
mol = Chem.MolFromSmiles(smiles)

# 添加氢原子
mol_with_hydrogen = Chem.AddHs(mol)

# 获取分子中的所有原子，包括氢原子
atoms = mol_with_hydrogen.GetAtoms()

# 遍历原子并打印每个原子的编号及其种类
for atom in atoms:
    atom_idx = atom.GetIdx()  # 获取原子编号
    atom_symbol = atom.GetSymbol()  # 获取原子种类
    print(f"Atom Index: {atom_idx}, Atom Type: {atom_symbol}")

In [ ]:
def CatNetmol_to_SMILES(filename):#将mollist.txt中的非标准SMILES转为标准的SMILES
    #得到分子数据的列表
    molecule_list = read_file_line_by_line(filename)
    mol_list=[]
    for mol in molecule_list:
        std_mol = add_brackets_around_letters(mol)
        mol_list.append(std_mol)
    return mol_list
CatNetmol_to_SMILES('mollist.txt')

In [ ]:
import numpy as np

# 创建两个相同形状的数组
array1 = np.array([1, 2, 3])
array2 = np.array([4, 5, 6])

# 直接相加
result = (array1 + array2)/2
print(result)  # 输出: [5 7 9]

In [13]:
from ase.build import fcc111, add_adsorbate
from ase.visualize import view
slab = fcc111('Al', size=(1,1,3))
z_max = max(slab.positions[:, 0])
positions = slab.get_positions()
print(positions)
for pos in enumerate(positions):
    # 检查位置是否超出限制
    print(pos)
#view(slab)


[[1.43189123 0.82670279 0.        ]
 [0.         1.65340558 2.33826859]
 [0.         0.         4.67653718]]
(0, array([1.43189123, 0.82670279, 0.        ]))
(1, array([0.        , 1.65340558, 2.33826859]))
(2, array([0.        , 0.        , 4.67653718]))


In [4]:
from ase.build import hcp0001, add_adsorbate
from ase.visualize import view
from ase import Atoms
from ase.io import read ,write
molecule = read('1.xyz')  
view(molecule)

<Popen: returncode: None args: ['c:\\Users\\renyq\\anaconda3\\envs\\ASE_env\...>

In [4]:
from ase.build import hcp0001
from ase.visualize import view
from ase.io import write
from ase.constraints import FixAtoms
import numpy as np
import os
from ase.io import write
from ase import Atoms
def build_slab(elment,size,vacuum,file_format,save_path,fix_num):
    x=size[0]
    y=size[1]
    z=size[2]
    ru_surface = hcp0001(elment, size=size, vacuum=vacuum)
    z_coords = ru_surface.positions[:, 2]
    fix_num=x*y*fix_num
    print(z_coords[fix_num])
    print(z_coords)
    fixed_indices = np.where(z_coords < z_coords[fix_num])[0]
    ru_surface.constraints = [FixAtoms(indices=fixed_indices)]
    file = elment+'_0001_surface.'+file_format
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    write(os.path.join(save_path, file), ru_surface)
    return ru_surface
slab = build_slab('Ru',(4,4,4),10,'cif','TEST',2)

14.276800000000001
[10.     10.     10.     10.     10.     10.     10.     10.     10.
 10.     10.     10.     10.     10.     10.     10.     12.1384 12.1384
 12.1384 12.1384 12.1384 12.1384 12.1384 12.1384 12.1384 12.1384 12.1384
 12.1384 12.1384 12.1384 12.1384 12.1384 14.2768 14.2768 14.2768 14.2768
 14.2768 14.2768 14.2768 14.2768 14.2768 14.2768 14.2768 14.2768 14.2768
 14.2768 14.2768 14.2768 16.4152 16.4152 16.4152 16.4152 16.4152 16.4152
 16.4152 16.4152 16.4152 16.4152 16.4152 16.4152 16.4152 16.4152 16.4152
 16.4152]


In [35]:
from ase import Atoms
from ase.io import read, write
from ase.geometry import find_mic
from ase.visualize import view
import numpy as np
import os
# 读取Ru(0001)表面的CIF文件
ru_surface = read('Ru_0001_surface.cif')

# 读取分子的XYZ文件
molecule = read('output_ase.xyz')

# 计算分子的结合位点
molecule_center = np.array([0,0,0])

# 找到Ru(0001)表面最上层原子的z坐标最大值
z_max = max(ru_surface.positions[:, 2])
print(z_max)
# 将分子的质心移动到这个高度
height_above_surface = 0.4# 可以根据需要调整高度
molecule.translate([0, 0, height_above_surface + z_max - molecule_center[2]])

# 将分子添加到表面上(PS:加法？！)
system = ru_surface + molecule
# 保存新的结构
view(system)
print("The system with the molecule placed on Ru(0001) surface has been written to Ru_0001_surface_with_molecule.cif")

16.4152
The system with the molecule placed on Ru(0001) surface has been written to Ru_0001_surface_with_molecule.cif


In [37]:
def check_dist_between_atoms(structure):
    cutoff_H = 0.5  # H
    cutoff_other = 1.0 #other atoms
    for i in range(len(structure)):
        for j in range(i+1, len(structure)):
            atom_i = structure[i]
            atom_j = structure[j]
            element_i = atom_i.symbol
            element_j = atom_j.symbol
            dist = structure.get_distances(i,j,mic=True)
            if element_i == 'H' or element_j == 'H':
                if dist < cutoff_H:
                    print(f'原子 {element_i}:{i} 和原子 {element_j}:{j} 之间的距离为 {dist}, 小于截断值 {cutoff_H}')
                    return False
                else:
                    pass
            else:
                if dist < cutoff_other:
                    print(f'原子 {element_i}:{i} 和原子 {element_j}:{j} 之间的距离为 {dist}埃, 小于截断值 {cutoff_other}')
                    return False
                else:
                    pass
    return True
check_dist_between_atoms(system)

原子 Ru:48 和原子 H:64 之间的距离为 [0.4], 小于截断值 0.5


False

In [ ]:
def place_mol_on_surface(mol,surface,shift_vector):
    # 找到Ru(0001)表面最上层原子的z坐标最大值
    z_max = max(surface.positions[:, 2])
    # 计算分子的结合位点,将分子的质心移动到这个高度
    molecule_center = shift_vector + np.array([0,0,z_max])
    mol.translate(molecule_center)
    # 将分子添加到表面上(PS:加法？！)
    system = surface + mol
    return system

In [4]:

def check_molecule_over_surface(surface,mol,sv):
    z_max = max(surface.positions[:, 2])
    molecule_center = sv + np.array([0,0,z_max])
    molcopy = copy.deepcopy(mol)
    molcopy.translate(molecule_center)
    z_min = min(molcopy.positions[:,2])
    if z_min >= z_max:
        return True
    else:
        return False
Ru_surface = hcp0001('Ru',size=(2,2,4),vacuum=10,periodic=True)
mol = Atoms('N2', positions=[[0, 0, 0], [0, 0, 1]])
sv = np.array([0,0,2])
check_molecule_over_surface(Ru_surface,mol,sv)


True

In [ ]:
from ase import Atoms
from ase.visualize import view
from ase.io import read, write
# 假设您已经有一个名为'molecule.xyz'的分子文件
molecule = read('./output/[O][C][O].traj')
view(molecule)
# 旋转分子，例如围绕z轴旋转90度
molecule.rotate(90, 'x', center=(0, 0, 0))

# 可视化旋转后的分子
view(molecule)

In [ ]:
from ase import Atoms
import copy
def rotate_mol(mol,angle,axis,rotatecenter):
    molcopy = copy.deepcopy(mol)
    molcopy.rotate(angle,axis,center=rotatecenter)
    return molcopy
    

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem

# 读取XYZ文件
def read_xyz(file_name):
    with open(file_name, 'r') as f:
        lines = f.readlines()
    atom_lines = lines[2:]  # 跳过前两行（通常包含原子数和标题）
    atoms = []
    for line in atom_lines:
        parts = line.split()
        atoms.append((parts[0], (float(parts[1]), float(parts[2]), float(parts[3]))))
    return atoms

# 将原子列表转换为RDKit分子对象
def atoms_to_mol(atoms):
    mol = Chem.RWMol()
    for atom, coords in atoms:
        mol.AddAtom(Chem.Atom(atom))
    conf = Chem.Conformer(len(atoms), coords)
    mol.AddConformer(conf)
    return Chem.Mol(mol)

# 读取XYZ文件
atoms = read_xyz('[H][C][H].xyz')

# 将原子列表转换为RDKit分子对象
mol = atoms_to_mol(atoms)

# 将分子对象转换为SMILES字符串
smiles = Chem.MolToSmiles(mol)
print(smiles)

In [ ]:
from ase.io import read
import SMILES_to_aseAtoms
# 假设你的XYZ文件名为'structure.xyz'
atoms = read('./output/[H][C]([H])([H])[O].xyz')
smiles = '[H][C]([H])([H])[O]'
smiles = SMILES_to_aseAtoms.enumerate_smiles(smiles)
print(smiles)
atom_types, adjacency_matrix,edges = SMILES_to_aseAtoms.smiles_to_chemical_graph(smiles)
print("原子种类:", atom_types)
print(len(atom_types))
print("邻接矩阵:")
print(adjacency_matrix)
print('bonds:',edges)
bondlist = SMILES_to_aseAtoms.count_nonzero_in_row(adjacency_matrix)
print(bondlist)
freelist=SMILES_to_aseAtoms.check_atoms_bonds(atom_types,bondlist)
print(freelist)
'''ghost atoms'''


C[O]
原子种类: [(0, 'C'), (1, 'O'), (2, 'H'), (3, 'H'), (4, 'H')]
5
邻接矩阵:
[[0 1 1 1 1]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]]
bonds: [(0, 1), (0, 2), (0, 3), (0, 4)]
[4 1 1 1 1]
[(1, 'O')]


'ghost atoms'

In [ ]:
from rdkit import Chem
import SMILES_to_aseAtoms
m = Chem.MolFromSmiles('[H][C]([H])[H]')  # 从SMILES字符串创建分子对象
mw = Chem.RWMol(m)  # 创建RWMol对象
new_atom_index = mw.AddAtom(Chem.Atom(0))  # 添加一个原子
mw.AddBond(1, new_atom_index, Chem.BondType.SINGLE)  # 在原子0和新添加的原子之间添加单键
#mw.RemoveAtom(0)  # 删除原子索引为0的原子
m_edit = mw.GetMol()  # 获取编辑后的分子对象
Chem.SanitizeMol(m_edit)  # 检查分子结构是否正确
print(Chem.MolToSmiles(m_edit))  # 输出分子的SMILES字符串

SMILES_to_aseAtoms_v101.check_input_SMILES(Chem.MolToSmiles(m_edit))

[16:00:36] 

****
Pre-condition Violation
attempt to add self-bond
Violation occurred on line 493 in file D:\bld\rdkit_1721396113642\work\Code\GraphMol\RWMol.cpp
Failed Expression: atomIdx1 != atomIdx2
****



RuntimeError: Pre-condition Violation
	attempt to add self-bond
	Violation occurred on line 493 in file Code\GraphMol\RWMol.cpp
	Failed Expression: atomIdx1 != atomIdx2
	RDKIT: 2024.03.5
	BOOST: 1_84


In [21]:
from rdkit import Chem
import re
def Bond_to_adGroup(smiles,n):
    m = Chem.MolFromSmiles(smiles)
    mw = Chem.RWMol(m)
    ghost_atom_index = mw.AddAtom(Chem.Atom(0))
    mw.AddBond(n, ghost_atom_index, Chem.BondType.SINGLE)
    m_edit = mw.GetMol()
    Chem.SanitizeMol(m_edit)
    new_ad_smiles = Chem.MolToSmiles(m_edit)
    return new_ad_smiles
smi = Bond_to_adGroup('[CH3]',0)
file_smi=re.sub(r"\*", "", smi)
print(smi,file_smi)

*C C


In [1]:
from ase.io import read, write
from ase.visualize import view
from ase import Atoms

# 创建一个示例原子结构
atoms = Atoms('N2', positions=[[0, 0, 0], [0, 0, 1]])
view(atoms, viewer='x3d')

In [7]:
from ase_notebook import AseView, ViewConfig, get_example_atoms
config = ViewConfig()
ase_view = AseView(config)
atoms = get_example_atoms()
ase_view.make_render(atoms, center_in_uc=True)

ModuleNotFoundError: No module named 'ase_notebook'

In [2]:
def remove_x_from_xyz(input_file, output_file):
    with open(input_file, 'r') as file:
        lines = file.readlines()

    # 新的原子列表，不包含"X"
    new_atoms = [line for line in lines if not line.strip().startswith('X')]

    # 写入新的xyz文件
    with open(output_file, 'w') as file:
        file.writelines(new_atoms)
remove_x_from_xyz('.\ot\[C]_ads_C.xyz','testC')

<>:11: SyntaxWarning: invalid escape sequence '\o'
<>:11: SyntaxWarning: invalid escape sequence '\o'
C:\Users\renyq\AppData\Local\Temp\ipykernel_6492\3839690866.py:11: SyntaxWarning: invalid escape sequence '\o'
  remove_x_from_xyz('.\ot\[C]_ads_C.xyz','testC')


In [5]:
from ase.io import read
from ase.atoms import Atoms

# 读取xyz文件
atoms = read('.\ot\[CH]([O])[O]_ads_C.xyz')

# 删除所有符号为"X"的原子
del atoms[[atom.index for atom in atoms if atom.symbol == 'X']]

# 将结果写入新的xyz文件
atoms.write('testC.xyz')

<>:5: SyntaxWarning: invalid escape sequence '\o'
<>:5: SyntaxWarning: invalid escape sequence '\o'
C:\Users\renyq\AppData\Local\Temp\ipykernel_6492\246912172.py:5: SyntaxWarning: invalid escape sequence '\o'
  atoms = read('.\ot\[CH]([O])[O]_ads_C.xyz')


In [16]:
def remove_placeholder_in_place(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # 移除以"X"开头的行
    new_lines = [line for line in lines if not line.strip().startswith('X')]

    # 写入修改后的内容回同一个文件
    with open(file_path, 'w') as file:
        file.writelines(new_lines)

# 使用函数
file_path = 'OC[O]_ads_O.xyz'  # xyz文件路径
remove_placeholder_in_place(file_path)

FileNotFoundError: [Errno 2] No such file or directory: 'OC[O]_ads_O.xyz'

In [ ]:
from ase import Atoms
from ase.geometry import distance

# 假设你已经有一个Atoms对象
# atoms = ...

# 定义一个阈值，例如0.1埃
threshold = 0.1

# 获取原子数量
num_atoms = len(atoms)

# 遍历所有原子对
for i in range(num_atoms):
    for j in range(i + 1, num_atoms):
        # 计算原子对之间的距离
        dist = distance(atoms.positions[i], atoms.positions[j], atoms.cell, atoms.pbc)
        
        # 判断距离是否小于阈值
        if dist < threshold:
            print(f"原子 {i} 和原子 {j} 重合")
        else:
            print(f"原子 {i} 和原子 {j} 不重合")


def check_distance_between_atoms(atoms,limit):
    num_atoms = len(atoms)
    for i in range(num_atoms):
        for j in range(i+1,num_atoms):
            dist = distance(atoms.positions[i], atoms.positions[j], atoms.cell, atoms.pbc)
        # 判断距离是否小于阈值
        if dist < limit:
            print(f"原子 {i} 和原子 {j} 重合")
            return False
        else:
            print(f"原子 {i} 和原子 {j} 不重合")
            return True

In [4]:
def txt_to_dict(filename, delimiter='=', comment_char='#'):
    dictionary = {}
    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()
            # 忽略空行和注释行
            if line and not line.startswith(comment_char):
                key, value = line.split(delimiter)
                dictionary[key.strip()] = value.strip()
    return dictionary

# 使用函数
filename = 'ASEtoadG_output\species_name.txt'  # 替换为你的文件路径
result_dict = txt_to_dict(filename)
print(result_dict)

<>:13: SyntaxWarning: invalid escape sequence '\s'
<>:13: SyntaxWarning: invalid escape sequence '\s'
C:\Users\renyq\AppData\Local\Temp\ipykernel_23964\1203745152.py:13: SyntaxWarning: invalid escape sequence '\s'
  filename = 'ASEtoadG_output\species_name.txt'  # 替换为你的文件路径
C:\Users\renyq\AppData\Local\Temp\ipykernel_23964\1203745152.py:13: SyntaxWarning: invalid escape sequence '\s'
  filename = 'ASEtoadG_output\species_name.txt'  # 替换为你的文件路径


ValueError: not enough values to unpack (expected 2, got 1)

In [11]:
def txt_to_dict(filename):
    dictionary = {}
    with open(filename, 'r') as file:
        for line in file:
            # 移除行首行尾的空白字符
            line = line.strip()
            # 忽略空行和注释行（假设注释行以'#'开头）
            if line and not line.startswith('#'):
                # 分割键和值，最多分割一次
                parts = line.split(':', 1)
                if len(parts) == 2:
                    key, value = parts
                    dictionary[key.strip()] = value.strip()
                else:
                    print(f"无法解析的行：{line}")
    return dictionary

# 使用函数
filename =  'ASEtoadG_output//species_name.txt'  # 替换为你的文件路径
result_dict = txt_to_dict(filename)
print(result_dict)
my_dict = result_dict
# 遍历字典的键
for key in my_dict:
    print(key)

# 遍历字典的值
for value in my_dict.values():
    print(value)

# 遍历字典的键值对
for key, value in my_dict.items():
    print(f"{key}: {value}")

{'*C': '-C_ads_C.xyz', '*OCO': '-OCO_ads_O.xyz', '*[CH]': '-[CH]_ads_C.xyz', '*OC[O]': '-OC[O]_ads_O.xyz', '*C([O])O': '-C([O])O_ads_C.xyz', '*O[CH]O': '-O[CH]O_ads_O.xyz', '*[C]': '-[C]_ads_C.xyz', 'OC': 'OC_ads_noBond_O.xyz', '*[C][O]': '-[C][O]_ads_C.xyz', '*O[C]': '-O[C]_ads_O.xyz', '*[CH2]': '-[CH2]_ads_C.xyz', '*[CH]O': '-[CH]O_ads_C.xyz', 'C': 'C_ads_noBond_C.xyz', '*CO': '-CO_ads_C.xyz', '*OC': '-OC_ads_O.xyz', '*C([O])[O]': '-C([O])[O]_ads_C.xyz', '*O[CH][O]': '-O[CH][O]_ads_O.xyz', '*C[O]': '-C[O]_ads_C.xyz', '*O[CH2]': '-O[CH2]_ads_O.xyz', '*[CH][O]': '-[CH][O]_ads_C.xyz', '*O[CH]': '-O[CH]_ads_O.xyz', '*O[C][O]': '-O[C][O]_ads_O.xyz', '*[C]([O])[O]': '-[C]([O])[O]_ads_C.xyz', '*[C]([O])O': '-[C]([O])O_ads_C.xyz', '*O[C]O': '-O[C]O_ads_O.xyz', '*[C]O': '-[C]O_ads_C.xyz'}
*C
*OCO
*[CH]
*OC[O]
*C([O])O
*O[CH]O
*[C]
OC
*[C][O]
*O[C]
*[CH2]
*[CH]O
C
*CO
*OC
*C([O])[O]
*O[CH][O]
*C[O]
*O[CH2]
*[CH][O]
*O[CH]
*O[C][O]
*[C]([O])[O]
*[C]([O])O
*O[C]O
*[C]O
-C_ads_C.xyz
-OCO_ads_O.xy

In [ ]:
def check_dist_between_atoms(structure):
    cutoff_H = 0.5  # H
    cutoff_other = 1.0 #other atoms
    for i in range(len(structure)):
        for j in range(i+1, len(structure)):
            atom_i = structure[i]
            atom_j = structure[j]
            element_i = atom_i.symbol
            element_j = atom_j.symbol
            dist = structure.get_distances(i,j,mic=True)
            if element_i == 'H' or element_j == 'H':
                if dist < cutoff_H:
                    print(f'原子 {element_i,i} 和原子 {element_j,j} 之间的距离为 {dist}, 小于截断值 {cutoff_H}')
                    return False
                else:
                    pass
            else:
                if dist < cutoff_other:
                    print(f'原子 {element_i,i} 和原子 {element_j,j} 之间的距离为 {dist}, 小于截断值 {cutoff_other}')
                    return False
                else:
                    pass
    return True

In [48]:
my_dict = {'a': 1, 'b': 2, 'c': 3}

# 获取所有值
values = my_dict.values()

# 将值转换为列表
values_list = list(my_dict.values)
print(values)
print(values_list)  # 输出: [1, 2, 3]


TypeError: 'builtin_function_or_method' object is not iterable

In [5]:
from ase.io import read
from ase import Atoms
from ase.build import molecule
from ase.cluster.icosahedron import Icosahedron
from ase.cluster.wulff import wulff_construction
s1 = read('./output//mol_to_ad//Ru--C//random_model1Ru--C.cif')
s2 = read('./output/mol_to_ad/Ru--C/random_model2Ru--C.cif')
rmsd= s1.get_rmsd(s2)
print(rmsd)

AttributeError: 'Atoms' object has no attribute 'get_rmsd'